In [2]:
import os
import numpy as np
import requests
import re
import json
from bs4 import BeautifulSoup

def get_last_part(url):
    return url.rstrip('/').split('/')[-1]

def get_tname(url):
    return "".join(url.rstrip('/').split('/')[-2:])

def filter_all_subsections(section_text):
    lines = section_text.split('\n')  # Split the content into lines
    subsections = []  # To store all subsections

    # This will hold the current subsection we're collecting
    current_subsection = []

    for line in lines:
        # Skip empty lines
        if line.strip() == "":
            continue
        
        # Check if it's a new subsection (first line is non-empty)
        if not current_subsection:
            current_subsection.append(line.strip())  # Add the first non-empty line of subsection
        elif '-' in line and line.strip() != "":
            # Add lines that contain "-" to the current subsection
            current_subsection.append(line.strip())
        else:
            # Once we encounter a non-dash line, store the current subsection and start a new one
            subsections.append("\n".join(current_subsection))
            current_subsection = [line.strip()]  # Start a new subsection with the next valid line

    # Don't forget to add the last collected subsection
    if current_subsection:
        subsections.append("\n".join(current_subsection))

    return subsections

def get_section_by_heading(url, keyword):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    # Find the first heading with the specified keyword (mappool or mappools)
    heading = soup.find(
        lambda tag: tag.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6'] and 
                    re.search(r"\bmappools?\b", tag.get_text(), re.IGNORECASE)
    )
    
    if not heading:
        return None  # No heading found with "mappool" or "mappools"

    # Get the level of the found heading (e.g., h2 => level 2, h3 => level 3)
    heading_level = int(heading.name[1])  # h2 -> 2, h3 -> 3, etc.
    content = []
    for sibling in heading.find_next_siblings():
        # Stop collecting when we encounter a heading of the same level or higher
        if sibling.name == heading.name:
            break
        a_tags = sibling.find_all('a', href=True)
        c = []
        if a_tags:
            for i in range(len(a_tags)):
                c.append(a_tags[i].get_text())
            content.append(c)
    #print("\n".join(content))
    #print("-----")
    flat = "\n".join([item for sublist in content for item in sublist])
    return flat

def get_slugs(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Get all visible text
    page_text = soup.get_text(separator="\n", strip=True)

    # Print all text
    #print("PAGE TEXT:")
    #print(page_text)
    
    # Get all hyperlinks (anchor tags)
    links = []
    for a_tag in soup.find_all("a"):
        if 'href' in a_tag.attrs:
            text = a_tag.get_text(strip=True)
            href = a_tag["href"]
            links.append((text, href))
        else:
            text = a_tag.get_text(strip=True)
            href = "https://osu.ppy.sh/b/0000000"
            links.apend((text, href))

    filtered_links = []
    for text, href in links:
        if pattern.match(href):
            last_segment = get_last_part(href)
            filtered_links.append((text, href, last_segment))

    # Tester output
    #for text, href, slug in filtered_links:
    #    print(f"Text: {text} | URL: {href} | ID/Slug: {slug}")
    
    slugs = []
    for text, href, slug in filtered_links:
        slugs.append(slug)
    return slugs

def generate_subsection_dict_n(subsection):
    lines = subsection.split('\n')  # Split the subsection into lines
    subsection_dict = {}

    # If the subsection has more than one line, start processing
    if len(lines) > 1:
        base_line = lines[0].strip()  # The first line (this will be the key base)
        # Add a counter to the first line
        for i, line in enumerate(lines[1:], start=1):  # Start from second line (index 1)
            subsection_dict[f"{base_line}{i}"] = {"meta": line.strip()}

    return subsection_dict

def generate_subsection_dict_slugs(subsection, slugs):
    lines = subsection.split('\n')  # Split the subsection into lines
    subsection_dict = {}

    # If the subsection has more than one line, start processing
    if len(lines) > 1:
        base_line = lines[0].strip()  # The first line (this will be the key base)
        # Add a counter to the first line
        for i, line in enumerate(lines[1:], start=1):  # Start from second line (index 1)
            subsection_dict[f"{base_line}{i}"] = {"id": slugs[i-1]}
    return subsection_dict

def make_tdic(filtered_sections, slugs, tname):
    stages = []
    stage_mods = []
    dicts = []
    dict_id = []
    current_stage_counter = 0

    slut_ctr = 0
    for subs in filtered_sections:
        if "\n" not in subs:
            if "Download" in subs or "VOD" in subs or "played" in subs or "This" in subs:
                pass
            else:
                stage_mods.append(current_stage_counter)
                current_stage_counter = 0
                stages.append(subs)
                
        else:
            b4 = slut_ctr
            temp = generate_subsection_dict_n(subs)
            slut_ctr += subs.count("\n")
            fuck = generate_subsection_dict_slugs(subs, slugs[b4:slut_ctr])
            current_stage_counter += 1
            dicts.append(temp)
            dict_id.append(fuck)

    stage_mods.append(current_stage_counter)
    stages.append(subs)

    #wow this is hacky fuck em up though
    del stage_mods[0]
    del stages[-1]

    tdic_names = {}
    tdic_slugs = {}
    stage_mods_cum = np.cumsum(stage_mods).tolist()
    stage_mods_cum.insert(0, 0)
    for i in range(len(stage_mods_cum)-1):
        #print(dicts[stage_mods_cum[i]:stage_mods_cum[i+1]][0])
        tdic_names[stages[i]] = dicts[stage_mods_cum[i]:stage_mods_cum[i+1]][0]
        tdic_slugs[stages[i]] = dict_id[stage_mods_cum[i]:stage_mods_cum[i+1]][0]
    dic_n, dic_s = {}, {}
    dic_n[tname] = tdic_names
    dic_s[tname] = tdic_slugs
    return dic_n, dic_s


pattern = re.compile(r"^https://osu\.ppy\.sh/(beatmapsets|b)/")

def main(url):
    # need to add null slugs when a map is missing a link.
    section_text = get_section_by_heading(url, "mappool")
    filtered_sections = filter_all_subsections(section_text)
    VAFslugs = get_slugs(url)
    tname = get_tname(url)
    print(filtered_sections)
    tdic_n, tdic_s = make_tdic(filtered_sections, VAFslugs, tname)
    fn = "./dbs/"+tname+".json"
    with open(fn, "w") as json_file:
        json.dump(tdic_s, json_file)

with open ("urls.txt", "r") as f:
    urls = f.readlines()
    urls = [line.strip() for line in urls]

main("https://osu.ppy.sh/wiki/en/Tournaments/AOTS/IDTS_1")
#fucking listerror is due to missing link
'''
for url in urls:
    try:
        main(url)
    except (IndexError, AttributeError) as e:
        print("Error url: {}".format(url))
'''
        
'''

# to merge all the json files

json_files = glob.glob("path/to/jsons/*.json")  # Adjust this to match your files

merged_data = {}

for file_path in json_files:
    with open(file_path, "r") as f:
        data = json.load(f)
        merged_data.update(data)  # Adds the single top-level key

# Save the merged result
with open("merged.json", "w") as f:
    json.dump(merged_data, f, indent=2)
    
'''




["Download the mappack here! (83 MB)\nMorimori Atsushi - Applique (TKS) [test]\nRemo Prototype[CV: Hanamori Yumiri] - Sendan Life (Y O U T A) [Inner Oni)]\nSara - Natu iro Present Wo BPM180 Ni Shitemita (asuasu_yura) [Special]\nyambabom - Over my head (Backfire) [Taikocalypse Act I : The Return]\nMorimori Atsushi - PUPA (TKS) [Inner Oni]\nThe Quick Brown Fox - The Big Black (MajorLeagueWobs Remix) (pmriva) [BESAR HITAM [NSC]]\nNekomata L.E.D.Master+ - Chrono Diver -PENDULUMs- (Nofool) [Hard Oni]\nLeaF - MEPHISTO (TKS) [TK's Inner Oni]\nKola Kid - can't hide your love (Ekoro) [Divine Light]\nOcelot - TSUBAKI (TKS) [TSUBAKI]\nA.SAKA - Nanatsu Issenzakura (tasuke912) [tasuke's Inner Oni]\nCi Mei Gui - Wu Xuan Lan (Nardoxyribonucleic) [Inner Oni]\nxi - ANiMA (tasuke912) [Inner Oni]\nyuzen - Ouka Ryouran (Charlotte) [Ryouran Oni]\nHate vs Brilliance - Qubism (Nishizumi) [Inner Oni]\nYuuyu - Iki o Koroshita Stokesia (Sayaka-) [Oni]", "Download the mappack here! (70 MB)\nMemme - Avalanche (Lo

'\n\n# to merge all the json files\n\njson_files = glob.glob("path/to/jsons/*.json")  # Adjust this to match your files\n\nmerged_data = {}\n\nfor file_path in json_files:\n    with open(file_path, "r") as f:\n        data = json.load(f)\n        merged_data.update(data)  # Adds the single top-level key\n\n# Save the merged result\nwith open("merged.json", "w") as f:\n    json.dump(merged_data, f, indent=2)\n\n'